In [1]:
!pip install -q sentence-transformers numpy pandas

In [2]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"harshiniryali","key":"575d0bfc943cb6254aff097326508a96"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d tunguz/big-five-personality-test
!unzip -q big-five-personality-test.zip -d bigfive

Dataset URL: https://www.kaggle.com/datasets/tunguz/big-five-personality-test
License(s): other
 89% 141M/159M [00:00<00:00, 1.48GB/s]
100% 159M/159M [00:00<00:00, 1.30GB/s]


In [6]:
!find bigfive -name "*.csv"

bigfive/IPIP-FFM-data-8Nov2018/data-final.csv


In [7]:
DATA_PATH= "bigfive/IPIP-FFM-data-8Nov2018/data-final.csv"

In [8]:
df= pd.read_csv(DATA_PATH, sep='\t')
print("Dataset shape:", df.shape)
df.head()

Dataset shape: (1015341, 110)


,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,...,2016-03-03 02:01:01,768.0,1024.0,9.0,234.0,6,1,GB,51.5448,0.1991
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,...,2016-03-03 02:01:20,1360.0,768.0,12.0,179.0,11,1,MY,3.1698,101.706
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,...,2016-03-03 02:01:56,1366.0,768.0,3.0,186.0,7,1,GB,54.9119,-1.3833
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,...,2016-03-03 02:02:02,1920.0,1200.0,186.0,219.0,7,1,GB,51.75,-1.25
4,3.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,3.0,4.0,...,2016-03-03 02:02:57,1366.0,768.0,8.0,315.0,17,2,KE,1.0,38.0


In [9]:
QUESTIONNAIRE = {
    "EXT": {
        "EXT1": "I am the life of the party.",
        "EXT2": "I feel comfortable around people.",
        "EXT3": "I start conversations.",
        "EXT4": "I talk to a lot of different people at parties.",
        "EXT5": "I don't mind being the center of attention.",
        "EXT6": "I make friends easily.",
        "EXT7": "I am skilled in handling social situations.",
        "EXT8": "I am outgoing and sociable.",
        "EXT9": "I enjoy social gatherings.",
        "EXT10": "I feel energized when interacting with others."
    },

    "EST": {
        "EST1": "I get stressed out easily.",
        "EST2": "I worry about things.",
        "EST3": "I feel anxious often.",
        "EST4": "I get upset easily.",
        "EST5": "I change my mood a lot.",
        "EST6": "I have frequent mood swings.",
        "EST7": "I get irritated easily.",
        "EST8": "I feel emotionally unstable.",
        "EST9": "I feel nervous in difficult situations.",
        "EST10": "I feel overwhelmed by stress."
    },

    "AGR": {
        "AGR1": "I am interested in people.",
        "AGR2": "I sympathize with others’ feelings.",
        "AGR3": "I have a soft heart.",
        "AGR4": "I take time out for others.",
        "AGR5": "I feel others' emotions.",
        "AGR6": "I make people feel at ease.",
        "AGR7": "I care about others.",
        "AGR8": "I am helpful and unselfish.",
        "AGR9": "I trust others easily.",
        "AGR10": "I am considerate."
    },

    "CSN": {
        "CSN1": "I am always prepared.",
        "CSN2": "I pay attention to details.",
        "CSN3": "I get chores done right away.",
        "CSN4": "I follow a schedule.",
        "CSN5": "I am exacting in my work.",
        "CSN6": "I complete tasks efficiently.",
        "CSN7": "I plan ahead.",
        "CSN8": "I am disciplined.",
        "CSN9": "I meet deadlines.",
        "CSN10": "I am reliable."
    },

    "OPN": {
        "OPN1": "I have a rich vocabulary.",
        "OPN2": "I have a vivid imagination.",
        "OPN3": "I have excellent ideas.",
        "OPN4": "I am quick to understand things.",
        "OPN5": "I enjoy abstract ideas.",
        "OPN6": "I enjoy artistic experiences.",
        "OPN7": "I value creativity.",
        "OPN8": "I like to explore new concepts.",
        "OPN9": "I enjoy philosophical discussions.",
        "OPN10": "I am curious about many things."
    }
}

In [10]:
user_answers = {}

print("Answer each question using numbers:")
print("1 = Strongly Disagree | 2 = Disagree | 3 = Neutral | 4 = Agree | 5 = Strongly Agree\n")

for trait, questions in QUESTIONNAIRE.items():
    print(f"\n========== {trait} TRAIT ==========")
    for idx, (qid, question) in enumerate(questions.items(), 1):
        while True:
            try:
                ans = int(input(f"Q{idx}. {question}\nYour response (1–5): "))
                if 1 <= ans <= 5:
                    user_answers[qid] = ans
                    break
            except:
                pass

Answer each question using numbers:
1 = Strongly Disagree | 2 = Disagree | 3 = Neutral | 4 = Agree | 5 = Strongly Agree


========== EXT TRAIT ==========
Q1. I am the life of the party.
Your response (1–5): 5
Q2. I feel comfortable around people.
Your response (1–5): 4
Q3. I start conversations.
Your response (1–5): 3
Q4. I talk to a lot of different people at parties.
Your response (1–5): 2
Q5. I don't mind being the center of attention.
Your response (1–5): 1
Q6. I make friends easily.
Your response (1–5): 5
Q7. I am skilled in handling social situations.
Your response (1–5): 4
Q8. I am outgoing and sociable.
Your response (1–5): 3
Q9. I enjoy social gatherings.
Your response (1–5): 2
Q10. I feel energized when interacting with others.
Your response (1–5): 1

========== EST TRAIT ==========
Q1. I get stressed out easily.
Your response (1–5): 5
Q2. I worry about things.
Your response (1–5): 4
Q3. I feel anxious often.
Your response (1–5): 3
Q4. I get upset easily.
Your response (1–5):

In [47]:
trait_scores = {
    'EXT': df[[c for c in df.columns if c.startswith('EXT')]].mean(axis=1).iloc[0],
    'EST': df[[c for c in df.columns if c.startswith('EST')]].mean(axis=1).iloc[0],
    'AGR': df[[c for c in df.columns if c.startswith('AGR')]].mean(axis=1).iloc[0],
    'CSN': df[[c for c in df.columns if c.startswith('CSN')]].mean(axis=1).iloc[0],
    'OPN': df[[c for c in df.columns if c.startswith('OPN')]].mean(axis=1).iloc[0]
}

for trait, questions in QUESTIONNAIRE.items():
    values = [user_answers[q] for q in questions]
    mean_score = np.mean(values)
    normalized = round((mean_score - 1) / 4, 3)
    trait_scores[trait] = normalized

In [48]:
def dominance_ranking(trait_scores):
    return sorted(trait_scores.items(), key=lambda x: x[1], reverse=True)

def stability_index(scores):
    return round(1 - np.var(list(scores.values())), 3)

def psychological_risk(scores):
    lows = sum(v < 0.35 for v in scores.values())
    if lows >= 3: return "High Psychological Risk"
    if lows == 2: return "Moderate Psychological Risk"
    return "Low Psychological Risk"

def recommendations(scores):
    top = dominance_ranking(scores)[0][0]
    return {
        "EXT": "Leadership and networking roles",
        "EST": "Stress management and emotional balance",
        "AGR": "People-oriented environments",
        "CSN": "Structured and goal-driven roles",
        "OPN": "Creative and research roles"
    }[top]

def careers(scores):
    top = dominance_ranking(scores)[0][0]
    return {
        "EXT": ["Marketing", "Sales"],
        "EST": ["Healthcare", "Counseling"],
        "AGR": ["HR", "Teaching"],
        "CSN": ["Engineering", "Accounting"],
        "OPN": ["Design", "Research"]
    }[top]


In [49]:
def build_user_text(user_answers):
    text = []
    for trait, questions in QUESTIONNAIRE.items():
        for qid, qtext in questions.items():
            text.append(f"{qtext} Answer: {user_answers[qid]}")
    return " ".join(text)

sbert = SentenceTransformer("paraphrase-MiniLM-L3-v2")

user_text = build_user_text(user_answers)
user_embedding = sbert.encode([user_text])

print("SBERT embedding shape:", user_embedding.shape)


SBERT embedding shape: (1, 384)


In [50]:
def trait_dominance(scores):
  return sorted(scores.items(), key=lambda x:x[1], reverse=True)

In [51]:
def stability_index(scores):
  return round(1 - np.var(list(scores.values())), 3)

In [52]:
def psychological_risk(scores):
  low = sum(v < 0.35 for v in scores.values())
  if low >=3: return "High Psychological Risk"
  if low == 2: return "Moderate Psychological Risk"
  return "Low Psychological Risk"

In [53]:
def confidence_weight(scores):
  output = {}
  for trait, val in scores.items():
    if val >= 0.7:
      output[trait] = "High"
    elif val >= 0.4:
      output[trait] = "Moderate"
    else:
      output[trait] = "Low"
  return output

In [54]:
def personality_risk_heat(scores):
  return{
      trait: "High" if val < 0.35 else "Low"
      for trait, val in scores.items()
  }

In [55]:
def trait_conflict(scores):
  conflicts = []

  if scores["EXT"] > 0.7 and scores["EST"] < 0.3:
    conflicts.append("Extroversion vs Emotional Stability")

  if scores["CSN"] > 0.7 and scores["OPN"] < 0.3:
    conflicts.append("Conscientiousness vs Openness")

  return conflicts if conflicts else ["No major conflicts detected"]

In [56]:
def personalized_recommendation(scores):
    max_trait = max(scores, key=scores.get)
    max_score = scores[max_trait]

    if max_score < 0.30:
        return (
            "Focus on emotional support, self-awareness, "
            "confidence building, and gradual personal development."
        )

    elif 0.30 <= max_score <0.60:
        return (
            "Engage in structured skill development, guided learning, "
            "and gradual responsibility-building activities."
        )

    else:
        trait_based_recommendations = {
            "EXT": "Engage in leadership, networking, and public-facing roles",
            "EST": "Practice stress-management and emotional regulation techniques",
            "AGR": "Collaborative and people-oriented environments suit you",
            "CSN": "Structured, detail-oriented, and goal-driven roles are ideal",
            "OPN": "Creative, research, and innovation-focused roles fit you well"
        }

    return trait_based_recommendations.get(
        max_trait,
        "Pursue balanced personal and professional development paths."
    )

In [57]:
def career_paths(scores):
    max_score = max(scores.values())

    if max_score < 0.30:
        return [
            "Personal development programs",
            "Counseling and mentoring",
            "Skill-building workshops"
        ]

    if 0.30 <= max_score < 0.60:
        return [
            "Structured training programs",
            "Apprenticeship roles",
            "Supervised entry-level positions"
        ]

    dominant = trait_dominance(scores)[0][0]
    trait_to_career = {
        "EXT": ["Marketing", "Sales", "Public Relations", "Management"],
        "EST": ["Healthcare", "Psychology"],
        "AGR": ["HR", "Teaching"],
        "CSN": ["Software Engineering", "Accounting"],
        "OPN": ["Research", "Design"]
    }

    return trait_to_career.get(
        dominant,
         ["General professional development"]
    )

In [58]:
from sklearn.metrics.pairwise import cosine_similarity

def explainability_output_sbirt(trait_scores):
  base = np.array(list(trait_scores.values())).reshape(1, -1)
  explain = {}

  for i, trait in enumerate(trait_scores):
    unit = np.zeros_like(base)
    unit[0, i] = 1
    explain[trait] = round(
        cosine_similarity(base, unit)[0, 0], 4
    )

  return explain

In [59]:
print("\nOVERALL PERSONALITY SUMMARY")
print("-" * 40)
print("\nDominant Traits:\n")
for t, v in dominance_ranking(trait_scores):
    print(f"{t}: {v*100:.2f}%")

print("\nStability Index:", stability_index(trait_scores))
print("\nPsychological Risk:", psychological_risk(trait_scores))
print("\nConfidence Weighted Interpretation:", confidence_weight(trait_scores))
print("\nRisk Heat Levels:", personality_risk_heat(trait_scores))
print("\nTrait Conflicts:", trait_conflict(trait_scores))
print("\nRecommendation:", personalized_recommendation(trait_scores))
print("\nCareer Paths:", career_paths(trait_scores))


OVERALL PERSONALITY SUMMARY
----------------------------------------

Dominant Traits:

EXT: 50.00%
EST: 50.00%
AGR: 50.00%
CSN: 50.00%
OPN: 50.00%

Stability Index: 1.0

Psychological Risk: Low Psychological Risk

Confidence Weighted Interpretation: {'EXT': 'Moderate', 'EST': 'Moderate', 'AGR': 'Moderate', 'CSN': 'Moderate', 'OPN': 'Moderate'}

Risk Heat Levels: {'EXT': 'Low', 'EST': 'Low', 'AGR': 'Low', 'CSN': 'Low', 'OPN': 'Low'}

Trait Conflicts: ['No major conflicts detected']

Recommendation: Engage in structured skill development, guided learning, and gradual responsibility-building activities.

Career Paths: ['Structured training programs', 'Apprenticeship roles', 'Supervised entry-level positions']


In [26]:
trait_items = {
    'EXT': [f'EXT{i}' for i in range(1, 11)],
    'EST': [f'EST{i}' for i in range(1, 11)],
    'AGR': [f'AGR{i}' for i in range(1, 11)],
    'CSN': [f'CSN{i}' for i in range(1, 11)],
    'OPN': [f'OPN{i}' for i in range(1, 11)],
}

In [27]:
trait_scores = {}

for trait, items in trait_items.items():
    trait_scores[trait] = df[items].mean(axis=1)

In [28]:
true_labels = {}
predicted_labels = {}

for trait in trait_scores:
    median = trait_scores[trait].median()
    true_labels[trait] = (trait_scores[trait] >= median).astype(int)
    predicted_labels[trait] = (trait_scores[trait] >= 3).astype(int)

In [29]:
from sklearn.metrics import accuracy_score

#traits = ["EXT", "EST", "AGR", "CSN", "OPN"]
accuracy_results = {}

for trait in trait_scores:
    acc = accuracy_score(true_labels[trait], predicted_labels[trait])
    accuracy_results[trait] = acc

In [30]:
print("Trait-wise Acccuracy:\n")
for trait, acc in accuracy_results.items():
    print(f"{trait}: {acc * 100:.2f}%")

overall_accuracy = sum(accuracy_results.values()) / len(accuracy_results)

print(f"\nOverall System Accuracy: {overall_accuracy * 100:.2f}%")

Trait-wise Acccuracy:

EXT: 100.00%
EST: 100.00%
AGR: 78.35%
CSN: 89.66%
OPN: 73.46%

Overall System Accuracy: 88.29%
